In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import paraguayfloodspy.visualize as viz # custom plotting library

Define parameters

In [ ]:
%matplotlib inline
savefig = False

Read in data

In [ ]:
wt = pd.read_csv('../data/derived/WeatherTypes.csv', parse_dates=True, index_col='time')
mjo = pd.read_csv('../data/accessed/daily_indices.csv', parse_dates=True, index_col='time')
enso = pd.read_csv('../data/accessed/monthly_indices.csv', parse_dates=True, index_col='time')
enso = enso.resample('1D').ffill()
mrg = wt.join(mjo).join(enso)
mrg.loc[mrg['amplitude'] < 1, 'phase'] =  0
mrg.head()

In [ ]:
mrg['enso_state'] = 'Neutral'
mrg.loc[mrg['nino_34'] < -1, 'enso_state'] = 'NINA'
mrg.loc[mrg['nino_34'] > 1, 'enso_state'] = 'NINO'

In [ ]:
conditional = mrg.groupby(['phase', 'enso_state'])['wtype'].apply(lambda g: g.value_counts()/len(g))
conditional = conditional.to_xarray()

In [ ]:
anomaly = conditional - conditional.mean(dim=['level_2'])

In [ ]:
p = anomaly.plot(col='enso_state', figsize=(15,5), cmap='PuOr')
plt.xticks([1,2,3,4,5,6])
for ax in p.axes.flat:
    ax.set_xlabel('Weather Type')
plt.show()